# 2. Mājasdarbs. Oļegs Korsaks 051RDB146
• Atrast datu kopu internetā. Piemēram no Awesomedata vai UCI Machine Learning

Repository vai citur

– Vēlams ar simtiem līdz tūkstošiem ierakstiem,

– Vēlams desmitiem atribūtiem,

– Skaitliskās vērtības atribūtos,

– Ar mērķa klasi/klasēm, bet var arī nepārtraukta vērtība,

– Dažām datu kopām var nebūt izvirzīta mērķa klase. Šādos gadījumos var izmantot vienu no atribūtiem par mērķa klasi (ja ir iespējams loģiski pamatot).

• Sadalīt datu kopu apmācības un testa kopās.

• Izveidot daudzslāņu mākslīgo neironu tīklu ar tensorflow un veikt apmācību.

• Aprēķināt un novērtēt apmācības un testa kopas kļūdu.

• Pamainīt tīkla konfigurāciju un novērtēt izmaiņu ietekmi

– Arhitektūru – slēpto slāņu skaits, neironu skaits slāņos,

– Aktivācijas funkcijas,

– Optimizācijas metodi.

In [1]:
import datetime

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')

if gpus:
    # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)

        # config = tf.compat.v1.ConfigProto()
        # config.gpu_options.per_process_gpu_memory_fraction = 0.2
        # config.gpu_options.allow_growth = True
        # session = tf.compat.v1.InteractiveSession(config=config)
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=2600)]
        )
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(f'{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs')
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

%load_ext tensorboard

sns.set_context('notebook', font_scale=2.0)
sns.set_style('darkgrid')

from imblearn.over_sampling import SMOTE
from matplotlib import pyplot as plt

%matplotlib inline

from sklearn.model_selection import train_test_split

2021-12-18 01:31:09.210849: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-18 01:31:09.251813: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-18 01:31:09.252165: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


1 Physical GPUs, 1 Logical GPUs


2021-12-18 01:31:09.271422: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-18 01:31:09.271939: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-18 01:31:09.272327: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-18 01:31:09.272658: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [2]:
SEED = 11
EPOCHS = 500

## Datu ielāde

In [3]:
data = pd.read_csv('data/winequality-white.csv', sep=';')

### Datu analīze

In [4]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


In [6]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
fixed acidity,4898.0,6.854788,0.843868,3.80000,6.300000,6.80000,7.3000,14.20000
volatile acidity,4898.0,0.278241,0.100795,0.08000,0.210000,0.26000,0.3200,1.10000
citric acid,4898.0,0.334192,0.121020,0.00000,0.270000,0.32000,0.3900,1.66000
residual sugar,4898.0,6.391415,5.072058,0.60000,1.700000,5.20000,9.9000,65.80000
chlorides,4898.0,0.045772,0.021848,0.00900,0.036000,0.04300,0.0500,0.34600
free sulfur dioxide,4898.0,35.308085,17.007137,2.00000,23.000000,34.00000,46.0000,289.00000
total sulfur dioxide,4898.0,138.360657,42.498065,9.00000,108.000000,134.00000,167.0000,440.00000
density,4898.0,0.994027,0.002991,0.98711,0.991723,0.99374,0.9961,1.03898
pH,4898.0,3.188267,0.151001,2.72000,3.090000,3.18000,3.2800,3.82000
sulphates,4898.0,0.489847,0.114126,0.22000,0.410000,0.47000,0.5500,1.08000


## Datu vizualizācija un analīze

Paskatīsimies, vai ir dati, kuru vērtības ļoti atšķiras no vairuma.

In [6]:
fig, ax = plt.subplots(ncols=6, nrows=2, figsize=(20, 10))
index = 0
ax = ax.flatten()

for col, value in data.items():
    sns.boxplot(y=col, data=data, ax=ax[index])
    index += 1

plt.tight_layout(pad=0.5, w_pad=0.0)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/kamikaze/local_projects/RTU/rtu-ai/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_24875/2053583454.py", line 9, in <module>
    plt.tight_layout(pad=0.5, w_pad=0.0)
  File "/home/kamikaze/local_projects/RTU/rtu-ai/venv/lib/python3.9/site-packages/matplotlib/pyplot.py", line 2302, in tight_layout
    return gcf().tight_layout(pad=pad, h_pad=h_pad, w_pad=w_pad, rect=rect)
  File "/home/kamikaze/local_projects/RTU/rtu-ai/venv/lib/python3.9/site-packages/matplotlib/figure.py", line 3197, in tight_layout
    kwargs = get_tight_layout_figure(
  File "/home/kamikaze/local_projects/RTU/rtu-ai/venv/lib/python3.9/site-packages/matplotlib/tight_layout.py", line 320, in get_tight_layout_figure
    kwargs = _auto_adjust_subplotpars(fig, renderer,
  File "/home/kamikaze/local_projects/RTU/rtu-ai/venv/lib/python3.9/site-packages/matp

TypeError: object of type 'NoneType' has no len()

Error in callback <function flush_figures at 0x7f20cd244ee0> (for post_execute):


KeyboardInterrupt: 

Dažiem (piem. "total sulfur dioxide") ir tādas vērtības. Varbūt tie ietekmēs precizitāti.

Paskatīsimies uz vērtību sadali.

In [ ]:
fig, ax = plt.subplots(ncols=6, nrows=2, figsize=(20, 10))
index = 0
ax = ax.flatten()

for col, value in data.items():
    sns.histplot(value, ax=ax[index])
    index += 1

plt.tight_layout(pad=0.5, w_pad=0.0)

Daži paramtetri satur ļoti lielas vērtības. Vajadzēs normalizēt.

In [ ]:
sns.countplot(x=data['quality'])

Bet "quality" gan ir cita lieta - mums pārsvārā ir vērtības 5, 6 un pat nav dažu vērtību vispār. Tāda klašu nelīdzsvarotība nozīmē, ka modelis varēs labi noklasificēt tikai 5. un 6. kvalitātes gadījumus. Tātad datus būs janobalansē.

### Paramtetru korelācija

In [ ]:
corr = data.corr()
plt.figure(figsize=(20, 10), dpi=92)
sns.heatmap(corr, annot=True, cmap='coolwarm')

Ir līdzīgi parametri tādi kā "free sulfur dioxide" in "total sulfur dioxide". Izskatas, ka viens ar otru pozitīvi korelē, jo ietilpst otrā. Diemžēl būs janoņem to kolonnu.

In [ ]:
samples = data.copy()
samples.pop('free sulfur dioxide')
qualities = samples.pop('quality')

### Klašu vērtību balansēšana

In [ ]:
qualities.value_counts()

In [ ]:
oversample = SMOTE(k_neighbors=4)
samples, qualities = oversample.fit_resample(samples, qualities)

In [ ]:
qualities.value_counts()

Dati tika sabalansēti.

In [ ]:
attr_count = len(samples.keys())
print(attr_count, len(qualities.value_counts()))

Mums ir 10 atribūti un 7 klases, jo nav datu priekš citām klasēm.

In [ ]:
def plot_history(history, metric='mse'):
    plt.figure(figsize=(18, 10), dpi=92)
    plt.plot(history.history['loss'])
    plt.plot(history.history[f'val_loss'])
    plt.title('model_loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train loss', 'validate loss'], loc='best')
    plt.show()

    plt.figure(figsize=(18, 10), dpi=92)
    plt.plot(history.history[metric])
    plt.plot(history.history[f'val_{metric}'])
    plt.title(f'model_{metric}')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend([f'train {metric}', f'validate {metric}'], loc='best')
    plt.show()

In [ ]:
def evaluate_model(model, samples, qualities, epochs=100, metric='mse'):
    samples_train, samples_test, qualities_train, qualities_test = train_test_split(samples, qualities, test_size=0.2,
                                                                                    random_state=SEED)

    with tf.device('/gpu:0'):
        samples_train = tf.constant(samples_train, dtype=tf.float64)
        samples_test = tf.constant(samples_test, dtype=tf.float64)
        qualities_train = tf.constant(qualities_train, dtype=tf.int64)
        qualities_test = tf.constant(qualities_test, dtype=tf.int64)

        log_dir = 'logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


        history = model.fit(samples_train, qualities_train, validation_data=(samples_test, qualities_test), shuffle=False,
                            epochs=epochs, batch_size=256, verbose=0, callbacks=[tensorboard_callback])
        plot_history(history, metric)
        result = model.predict(samples_test)

        for sample_idx in (-5, -10, -20):
            print(f'Predicted: {result[sample_idx][0]}, Expected: {qualities_test[sample_idx]}')

In [ ]:
def build_model(input_count: int, metric='mse'):
    binit = tf.keras.initializers.Zeros()
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(input_count),
        tf.keras.layers.Dense(1, bias_initializer=binit)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

    model.compile(loss='mae', optimizer=optimizer, metrics=[metric])

    return model

In [ ]:
%%time
model = build_model(attr_count)
evaluate_model(model, samples, qualities, epochs=EPOCHS*2)

Nu vispār nav tik slikti :)) Pamēģināšu to pašu modeli ar x10 lielāko learning rate.

In [ ]:
def build_model(input_count: int, metric='mse'):
    binit = tf.keras.initializers.Zeros()
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(input_count),
        tf.keras.layers.Dense(1, bias_initializer=binit)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss='mae', optimizer=optimizer, metrics=[metric])

    return model

In [ ]:
%%time
model = build_model(attr_count)
evaluate_model(model, samples, qualities, epochs=EPOCHS*2)

Tas pats loss tika iegūts atrāk, pēc 200-300 epohām. Pievienosim vienu slāni ar vienu neironu.

In [ ]:
def build_model(input_count: int, metric='mse'):
    kinit = tf.keras.initializers.RandomNormal(stddev=0.05, seed=SEED)
    binit = tf.keras.initializers.Zeros()
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(input_count),
        tf.keras.layers.Dense(1, kernel_initializer=kinit, bias_initializer=binit, activation='relu'),
        tf.keras.layers.Dense(1, bias_initializer=binit)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss='mae', optimizer=optimizer, metrics=[metric])

    return model

In [ ]:
%%time
model = build_model(attr_count)
evaluate_model(model, samples, qualities, epochs=EPOCHS*2)

Izskatās, ka palika sliktāk nekā vispār bez neironiem. Pēc apm. 50. epohas progress apstājas. Palielināšu neironu skaitu līdz 2, skatīšos, vai kaut kas uzlabosies...

In [ ]:
def build_model(input_count: int, metric='mse'):
    kinit = tf.keras.initializers.RandomNormal(stddev=0.05, seed=SEED)
    binit = tf.keras.initializers.Zeros()
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(input_count),
        tf.keras.layers.Dense(2, kernel_initializer=kinit, bias_initializer=binit, activation='relu'),
        tf.keras.layers.Dense(1, bias_initializer=binit)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss='mae', optimizer=optimizer, metrics=[metric])

    return model

In [ ]:
%%time
model = build_model(attr_count)
evaluate_model(model, samples, qualities, epochs=EPOCHS*2)

Šitais modelis uzvedas interesanti. Vispirms viņš sasniedza to pašu loss līmeni ka pirmas (bez neironiem) modelis bet jau pēc 50. epohas. Un itkā progress apstājas... bet pie 400. epohas loss atkal nedaudz samazinājās. Palielināšu learning rate x5.

In [ ]:
def build_model(input_count: int, metric='mse'):
    kinit = tf.keras.initializers.RandomNormal(stddev=0.05, seed=SEED)
    binit = tf.keras.initializers.Zeros()
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(input_count),
        tf.keras.layers.Dense(2, kernel_initializer=kinit, bias_initializer=binit, activation='relu'),
        tf.keras.layers.Dense(1, bias_initializer=binit)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)

    model.compile(loss='mae', optimizer=optimizer, metrics=[metric])

    return model

In [ ]:
%%time
model = build_model(attr_count)
evaluate_model(model, samples, qualities, epochs=EPOCHS*2)

Kļūda raustas vairāk, rezultāts tas pats. Palielināšu neironu skaitu līdz 8.

In [ ]:
def build_model(input_count: int, metric='mse'):
    kinit = tf.keras.initializers.RandomNormal(stddev=0.05, seed=SEED)
    binit = tf.keras.initializers.Zeros()
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(input_count),
        tf.keras.layers.Dense(8, kernel_initializer=kinit, bias_initializer=binit, activation='relu'),
        tf.keras.layers.Dense(1, bias_initializer=binit)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)

    model.compile(loss='mae', optimizer=optimizer, metrics=[metric])

    return model

In [ ]:
%%time
model = build_model(attr_count)
evaluate_model(model, samples, qualities, epochs=EPOCHS*2)

Nu nekādi nevar pietuvināties pie 9. Palielināšu learning rate x20.

In [ ]:
def build_model(input_count: int, metric='mse'):
    kinit = tf.keras.initializers.RandomNormal(stddev=0.05, seed=SEED)
    binit = tf.keras.initializers.Zeros()
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(input_count),
        tf.keras.layers.Dense(8, kernel_initializer=kinit, bias_initializer=binit, activation='relu'),
        tf.keras.layers.Dense(1, bias_initializer=binit)
    ])

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

    model.compile(loss='mae', optimizer=optimizer, metrics=[metric])

    return model

In [ ]:
%%time
model = build_model(attr_count)
evaluate_model(model, samples, qualities, epochs=EPOCHS*2)